In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import networkx as nx
from numba import cuda
import time
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(".\\lasftm_asia\\lastfm_asia_edges.csv")

In [4]:
df.head()

,node_1,node_2
0,0,747
1,1,4257
2,1,2194
3,1,580
4,1,6478


In [ ]:
df.shape[0] # number of edges

27806

In [6]:
# making the graph
G = nx.from_pandas_edgelist(df, source='node_1', target='node_2')

In [ ]:
# draw the graph
plt.figure(figsize=(100, 50))
nx.draw(
    G,
    with_labels=False,
    node_color='red',
    edge_color='black',
    node_size=2000,
    font_size=8
)
plt.title("Graph of LastFM User Network")
plt.show()

In [9]:
def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

In [11]:
adj_lastfm = nx.to_numpy_array(G, dtype=np.float32)

print("Numpy Adjacency Matrix:\n", adj_lastfm)

Numpy Adjacency Matrix:
 [[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
def Normalize(matrix):
    matrix2 = matrix.copy()

    for j in range(len(matrix2[0])):
        temp = 0
        for i in range(len(matrix2)):
            temp += matrix2[i][j]
        for i in range(len(matrix2)):
            matrix2[i][j] = matrix2[i][j] / temp
        
    return matrix2

def Inflate(matrix, r):
    matrix = cp.power(matrix, r)
    col_sums = cp.sum(matrix, axis=0)
    matrix = matrix / col_sums
    return matrix

@cuda.jit
def matmul(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

def powerMatrix(matrix, n, bpg, tpb):
    if n < 1:
        raise ValueError("Pangkat matrix harus >= 1")

    matrix1 = matrix
    matrix2 = cp.zeros_like(matrix)

    for i in range(n - 1):
        matmul[bpg, tpb](matrix1, matrix, matrix2)
        cuda.synchronize()  # penting untuk sinkronisasi kernel
        matrix1 = matrix2.copy()

    return matrix1

def print_matrix(matrix):
    for row in matrix:
        print(" ".join(f"{val:.2f}" for val in row))

def MarkovCluster(matrix_a, e, r, iter):
    matrix = matrix_a

    # Add self-loops
    for i in range(len(matrix)):
        matrix[i][i] = 1

    matrix = Normalize(matrix)

    print("Normalized Matrix:\n", matrix)

    matrix = cp.asarray(matrix)

    # Setup CUDA kernel execution config
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(matrix.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(matrix.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    iterate = iter
    tol = 1e-5
    ptr = 0
    converged = False

    while ptr < iterate and not converged:
        prev_matrix = matrix.copy()

        matrix = powerMatrix(matrix, e, bpg=blockspergrid, tpb=threadsperblock)
        matrix = Inflate(matrix, r)

        # Cek konvergensi
        diff = cp.abs(matrix - prev_matrix)
        max_diff = cp.max(diff)
        if max_diff < tol:
            converged = True

        ptr += 1

    matrix = matrix.get()
    return matrix

In [18]:
e_1 = 2
r_1 = 2
start = time.time()
MarkovMatrix = MarkovCluster(matrix_a=adj_lastfm, e = e_1, r = r_1, iter = 10)
end = time.time()

Normalized Matrix:
 [[0.5        0.11111111 0.         ... 0.         0.         0.        ]
 [0.5        0.11111111 0.         ... 0.         0.         0.        ]
 [0.         0.         0.09090909 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.5        0.         0.        ]
 [0.         0.         0.         ... 0.         0.5        0.        ]
 [0.         0.         0.         ... 0.         0.         0.5       ]]


In [20]:
print(MarkovMatrix)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.3000930e-10 1.6755864e-11 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


In [21]:
def get_clusters_from_square_matrix(mcl_matrix):
    """
    mcl_matrix: square numpy array (n_nodes x n_nodes), hasil dari MCL
                dengan 1s menunjukkan koneksi antar node dalam cluster
    return: list of clusters, each cluster is a list of node indices
    """
    # Buat graph dari adjacency matrix
    G = nx.from_numpy_array(mcl_matrix)

    # Ambil connected components (cluster)
    clusters = [list(component) for component in nx.connected_components(G)]

    return clusters

In [22]:
clusters = get_clusters_from_square_matrix(MarkovMatrix)

for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")

Cluster 1: [0, 1, 3, 4, 6, 9, 10, 14, 21, 22, 25, 26, 29, 31, 33, 35, 36, 38, 40, 41, 42, 43, 45, 46, 47, 48, 49, 51, 56, 60, 64, 65, 66, 67, 69, 72, 73, 74, 75, 76, 77, 78, 79, 80, 86, 87, 88, 89, 90, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139, 140, 141, 143, 144, 145, 147, 148, 149, 150, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 178, 179, 182, 189, 190, 193, 194, 197, 199, 200, 202, 206, 208, 210, 214, 215, 216, 217, 218, 219, 220, 221, 224, 225, 230, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 250, 251, 253, 254, 255, 257, 258, 260, 261, 263, 265, 266, 267, 268, 269, 271, 272, 273, 274, 276, 281, 282, 283, 286, 288, 291, 292, 295, 296, 297, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 316, 317, 320, 323, 325, 